In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import MobileNetV2

In [3]:
# Paths
ANNOTATIONS = Path(r"A:\cats_dogs_light\Proiect RN COMARDICI ALEXANDRU\dataset\annotations.csv")
IMAGES_DIR = Path(r"A:\cats_dogs_light\Proiect RN COMARDICI ALEXANDRU\dataset\train")
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 30
MODEL_SAVE_PATH = Path(r"A:\cats_dogs_light\Proiect RN COMARDICI ALEXANDRU\dataset\animal_detector.keras")

In [4]:
# Load CSV
df = pd.read_csv(ANNOTATIONS)

In [ ]:
# Encode columns
df['species_enc'] = df['species'].map({'cat':0, 'dog':1})
size_classes = ['short','medium','long']
df['size_enc'] = df['size'].apply(lambda s: [1 if s==c else 0 for c in size_classes])

In [9]:
# Train/validation split
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['species_enc'], random_state=42)

In [10]:
# Helper to load images + labels
def df_to_numpy(df, img_dir):
    X, y_species, y_owner, y_size = [], [], [], []
    for _, row in df.iterrows():
        img_path = img_dir / row['filename']
        img = image.load_img(img_path, target_size=IMG_SIZE)
        arr = image.img_to_array(img)/255.0
        X.append(arr)
        y_species.append(to_categorical(row['species_enc'], 2))
        y_owner.append(np.array([row['has_owner']], dtype=np.float32))
        y_size.append(np.array(row['size_enc'], dtype=np.float32))
    return np.array(X), {'species': np.array(y_species), 
                         'has_owner': np.array(y_owner), 
                         'size': np.array(y_size)}

In [11]:
X_train, y_train_dict = df_to_numpy(train_df, IMAGES_DIR)
X_val, y_val_dict = df_to_numpy(val_df, IMAGES_DIR)

KeyboardInterrupt: 

In [12]:
# Model
base = MobileNetV2(input_shape=IMG_SIZE+(3,), include_top=False, weights='imagenet')
base.trainable = False

inputs = layers.Input(shape=IMG_SIZE+(3,))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)

species_out = layers.Dense(2, activation='softmax', name='species')(x)
owner_out = layers.Dense(1, activation='sigmoid', name='has_owner')(x)
size_out = layers.Dense(3, activation='softmax', name='size')(x)

model = models.Model(inputs=inputs, outputs=[species_out, owner_out, size_out])
model.compile(
    optimizer='adam',
    loss={'species':'categorical_crossentropy',
          'has_owner':'binary_crossentropy',
          'size':'categorical_crossentropy'},
    metrics={'species':'accuracy','has_owner':'accuracy','size':'accuracy'}
)

In [13]:
sample_weight_train = {
    'species': np.ones(len(train_df), dtype=np.float32),
    'has_owner': np.ones(len(train_df), dtype=np.float32),
    'size': np.ones(len(train_df), dtype=np.float32)
}

sample_weight_val = {
    'species': np.ones(len(val_df), dtype=np.float32),
    'has_owner': np.ones(len(val_df), dtype=np.float32),
    'size': np.ones(len(val_df), dtype=np.float32)
}

# Callbacks
cb = [
    callbacks.ModelCheckpoint(
        MODEL_SAVE_PATH,
        save_best_only=True,
        monitor='val_species_accuracy',
        mode='max'
    )
]

In [27]:
# Training
history = model.fit(
    X_train,
    y_train_dict,
    validation_data=(X_val, y_val_dict),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=cb
)


Epoch 1/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 16s 228ms/step - has_owner_accuracy: 0.7693 - has_owner_loss: 0.4579 - loss: 1.2247 - size_accuracy: 0.7553 - size_loss: 0.6092 - species_accuracy: 0.9379 - species_loss: 0.1561 - val_has_owner_accuracy: 0.7417 - val_has_owner_loss: 0.4603 - val_loss: 1.1619 - val_size_accuracy: 0.7417 - val_size_loss: 0.6458 - val_species_accuracy: 0.9801 - val_species_loss: 0.0726
Epoch 2/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 11s 203ms/step - has_owner_accuracy: 0.8560 - has_owner_loss: 0.3285 - loss: 0.7686 - size_accuracy: 0.8302 - size_loss: 0.3893 - species_accuracy: 0.9836 - species_loss: 0.0426 - val_has_owner_accuracy: 0.7748 - val_has_owner_loss: 0.4219 - val_loss: 0.9986 - val_size_accuracy: 0.7881 - val_size_loss: 0.4978 - val_species_accuracy: 0.9669 - val_species_loss: 0.0802
Epoch 3/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 11s 197ms/step - has_owner_accuracy: 0.8759 - has_owner_loss: 0.2772 - loss: 0.6092 - size_accuracy: 0.8677 - size_loss: 0.3104 - species_accuracy: 

In [28]:
model.evaluate(X_val,y_val_dict)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 346ms/step - has_owner_accuracy: 0.7748 - has_owner_loss: 0.8802 - loss: 2.0676 - size_accuracy: 0.7616 - size_loss: 1.0605 - species_accuracy: 0.9735 - species_loss: 0.1144


[2.0675716400146484,
 0.1143902912735939,
 0.880168080329895,
 1.0605101585388184,
 0.7748344540596008,
 0.7615894079208374,
 0.9735099077224731]

In [ ]:
def predict_image(model, img_path):
    """
    Primește modelul și calea către o imagine, returnează predicția pentru acea imagine.
    """
    img = image.load_img(img_path, target_size=IMG_SIZE)
    arr = image.img_to_array(img) / 255.0
    arr = np.expand_dims(arr, 0)
    
    species_pred, owner_pred, size_pred = model.predict(arr, verbose=0)
    
    species_label = 'dog' if np.argmax(species_pred[0]) == 1 else 'cat'
    owner_prob = float(owner_pred[0][0])
    size_label = ['short', 'medium', 'long'][int(np.argmax(size_pred[0]))]
    
    # dacă e pisică, ignoră size
    if species_label == 'cat':
        size_label = None
    
    return {'species': species_label, 'has_owner_prob': owner_prob, 'size': size_label}

In [15]:
example_filename = df['filename'].iloc[7]  # ia prima imagine din CSV
result = predict_image(model, IMAGES_DIR / example_filename)
print("Predicție din CSV:", result)

Predicție din CSV: {'species': 'dog', 'has_owner_prob': 0.26523563265800476, 'size': 'medium'}


In [16]:
folder = IMAGES_DIR / "dog"  # sau "cat_small", după caz
example_file = next(folder.glob("dog10.jpg"))  # ia primul fișier jpg
result2 = predict_image(model, example_file)
print("Predicție din folder:", result2)

Predicție din folder: {'species': 'dog', 'has_owner_prob': 0.44893679022789, 'size': 'big'}
